

> Import libraries to use



In [ ]:
import numpy as np

>  # Introduction to numpy (Skip if you already are familiar)

>> Creating a 1D array

In [ ]:
a = np.array([1,2,3,4])
print(a)

[1 2 3 4]


>> Creating a 2D array


In [ ]:
a = np.array([[1,2],[3,4]])
print(a)

[[1 2]
 [3 4]]


>> Creating an array full of zeros


In [ ]:
a = np.zeros(shape=(10))
print(a)
a = np.zeros(shape=(5,2))
print(a)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


>> Infinity in numpy

In [ ]:
print(np.inf)

inf


>> Max and Argmax

In [ ]:
a = np.array([2,1,4,3])
print(np.max(a))
print(np.argmax(a))

4
2


>> From list to Numpy

In [ ]:
l = [1,2,3,4]
print(l)
print(np.asarray(l))

[1, 2, 3, 4]
[1 2 3 4]


>> Random in numpy

In [ ]:
# Array of Random integers ranging from 1 to 10 (with any size you want)
a = np.random.randint(low=1, high=10, size=(5,2))
print(a)

# Array of random elements of a list with any size you want
a = np.random.choice([0,1,2], size=(2,))

[[6 4]
 [1 6]
 [7 5]
 [5 9]
 [4 2]]


>> Shapes in numpy

In [ ]:
a = np.random.randint(low=1, high=5, size=(4,2))
print(a.shape)
print(a)

# Reshape a to a vector of shape = (8,1)
a = a.reshape((8,1))
print(a.shape)
print(a)

(4, 2)
[[4 3]
 [3 2]
 [2 3]
 [2 2]]
(8, 1)
[[4]
 [3]
 [3]
 [2]
 [2]
 [3]
 [2]
 [2]]


# Pre-defined utilities

In [ ]:

int_to_char = {
    0 : 'u',
    1 : 'r',
    2 : 'd',
    3 : 'l'
}

policy_one_step_look_ahead = {
    0 : [-1,0],
    1 : [0,1],
    2 : [1,0],
    3 : [0,-1]
}

def policy_int_to_char(pi,n):

    pi_char = ['']

    for i in range(n):
        for j in range(n):

            if i == 0 and j == 0 or i == n-1 and j == n-1:

                continue

            pi_char.append(int_to_char[pi[i,j]])

    pi_char.append('')

    return np.asarray(pi_char).reshape(n,n)

# 1- Policy evaluation

In [9]:
def policy_evaluation(n, pi, v, Gamma, threshhold, max_iterations=1000):
    """
    This function should return the value function that follows the policy pi.
    Use the stopping criteria given in the problem statement.
    """
    iteration = 0
    while True:
        delta = 0
        v_new = v.copy()
        for i in range(n):
            for j in range(n):
                if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
                    continue  # Skip terminal states
                a = pi[i, j]  # Current action from policy
                # Determine next state based on action
                if a == 0:  # UP
                    i_new = max(i - 1, 0)
                    j_new = j
                elif a == 1:  # RIGHT
                    i_new = i
                    j_new = min(j + 1, n - 1)
                elif a == 2:  # DOWN
                    i_new = min(i + 1, n - 1)
                    j_new = j
                elif a == 3:  # LEFT
                    i_new = i
                    j_new = max(j - 1, 0)
                else:
                    raise ValueError("Invalid action")
                reward = -1  # Reward per step
                v_new[i, j] = reward + Gamma * v[i_new, j_new]
                delta = max(delta, abs(v_new[i, j] - v[i, j]))
        v = v_new.copy()
        iteration += 1
        if delta <= threshhold or iteration >= max_iterations:
            if iteration >= max_iterations:
                print(f"Policy evaluation reached maximum iterations ({max_iterations}) without full convergence.")
            break
    return v

# 2- Policy improvement

In [10]:
def policy_improvement(n, pi, v, Gamma):
    """
    This function should return the new policy by acting in a greedy manner.
    The function should return as well a flag indicating if the output policy
    is the same as the input policy.

    Example:
      return new_pi, True if new_pi = pi for all states
      else return new_pi, False
    """
    policy_stable = True
    new_pi = pi.copy()
    for i in range(n):
        for j in range(n):
            if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
                continue  # Skip terminal states
            old_action = pi[i, j]
            action_values = []
            for a in range(4):
                # Determine next state based on action
                if a == 0:  # UP
                    i_new = max(i - 1, 0)
                    j_new = j
                elif a == 1:  # RIGHT
                    i_new = i
                    j_new = min(j + 1, n - 1)
                elif a == 2:  # DOWN
                    i_new = min(i + 1, n - 1)
                    j_new = j
                elif a == 3:  # LEFT
                    i_new = i
                    j_new = max(j - 1, 0)
                reward = -1  # Reward per step
                value = reward + Gamma * v[i_new, j_new]
                action_values.append(value)
            best_action = np.argmax(action_values)
            new_pi[i, j] = best_action
            if old_action != best_action:
                policy_stable = False
    return new_pi, policy_stable


# 3- Policy Initialization

In [11]:
def policy_initialization(n):
    """
    This function should return the initial random policy for all states.
    """
    pi = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
                pi[i, j] = -1  # Terminal states
            else:
                pi[i, j] = np.random.choice([0, 1, 2, 3])
    return pi


# 4- Policy Iteration algorithm

In [12]:
def policy_iteration(n, Gamma, threshhold, max_iterations=1000):
    pi = policy_initialization(n=n)
    v = np.zeros(shape=(n, n))
    iteration = 0
    while True:
        v = policy_evaluation(n=n, v=v, pi=pi, threshhold=threshhold, Gamma=Gamma, max_iterations=max_iterations)
        pi, pi_stable = policy_improvement(n=n, pi=pi, v=v, Gamma=Gamma)
        iteration += 1
        if pi_stable or iteration >= max_iterations:
            if iteration >= max_iterations:
                print(f"Policy iteration reached maximum iterations ({max_iterations}) without policy stability.")
            break
    return pi, v

# Main Code to Test

In [ ]:
n = 4
Gamma = [0.8, 0.9, 1]
threshhold = 1e-4

action_map = {0: 'U', 1: 'R', 2: 'D', 3: 'L', -1: '-'}

for _gamma in Gamma:
    pi, v = policy_iteration(n=n, Gamma=_gamma, threshhold=threshhold, max_iterations=1000)

    print()
    print("Gamma = ", _gamma)
    print()
    print("Optimal Policy:")
    for i in range(n):
        for j in range(n):
            print(action_map.get(pi[i, j], '?'), end=' ')
        print()
    print()
    print("Value Function:")
    print(v)
#Je n'arrive pas à obtenir un résultat pour gamma = 1
#Lorsque je ne met pas de limite d'itération le code tourne en boucle


Gamma =  0.8

Optimal Policy:
- L L D 
U U U D 
U U R D 
U R R - 

Value Function:
[[ 0.   -1.   -1.8  -2.44]
 [-1.   -1.8  -2.44 -1.8 ]
 [-1.8  -2.44 -1.8  -1.  ]
 [-2.44 -1.8  -1.    0.  ]]

Gamma =  0.9

Optimal Policy:
- L L D 
U U U D 
U U R D 
U R R - 

Value Function:
[[ 0.   -1.   -1.9  -2.71]
 [-1.   -1.9  -2.71 -1.9 ]
 [-1.9  -2.71 -1.9  -1.  ]
 [-2.71 -1.9  -1.    0.  ]]
Policy evaluation reached maximum iterations (1000) without full convergence.
Policy evaluation reached maximum iterations (1000) without full convergence.
Policy evaluation reached maximum iterations (1000) without full convergence.

Gamma =  1

Optimal Policy:
- L L D 
U U U D 
U U R D 
U R R - 

Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
